## Realistic Power Spectrum

In this section, we attempt to derive a realistic power spectrum. We use the angular power spectrum of Blake+2008, $C_l = 2\times 10^{-4} l^{-1.2}$, and from it derive an approximate power spectrum in $u,v$, fitting a power-law power spectrum form to the result/.

In [1]:
from healpy import sphtfunc
from healpy import visufunc
from healpy import pixelfunc
from scipy.interpolate import griddata
from scipy.stats import linregress
from powerbox.tools import get_power

ImportError: No module named healpy

In [ ]:
## Set up grid of l, and C.
l = np.arange(1,1000)
cls = 2e-4 * l**-1.2

# Generate the map from cls.
clmap = sphtfunc.synfast(cls,64)

# Find co-latitude and longitude of map pixels.
theta, phi = pixelfunc.pix2ang(64,np.arange(12*64**2))

In [ ]:
# Restrict map to a single hemisphere.
clmap_hemi = clmap[theta<=np.pi/2]
phi = phi[theta<=np.pi/2]
theta = theta[theta<=np.pi/2]

In [ ]:
# Generate semi-Euclidean l, m co-ordinates
l = np.sin(theta)*np.cos(phi)
m = np.sin(theta)*np.sin(phi)

In [ ]:
# Generate a uniform l,m grid. Only go up to 1/sqrt(2), so that the square stays within the circle.
lgrid = np.linspace(-1/np.sqrt(2),1/np.sqrt(2),3000)
L,M = np.meshgrid(lgrid,lgrid)

# Interpolate map onto the uniform grid.
interp_map = griddata(np.array([l,m]).T,clmap_hemi, (L,M))
interp_map[np.isnan(interp_map)] = np.nanmean(interp_map)

In [ ]:
# Make a plot of the map
plt.imshow(interp_map,extent=[-1/np.sqrt(2),1/np.sqrt(2),-1/np.sqrt(2),1/np.sqrt(2)])

In [ ]:
# Generate the power spectrum of the map
pk, k = get_power(interp_map,2/np.sqrt(2))

In [ ]:
# Find the slope and intercept of the resulting power spectrum
slope, intercept, _,_,_ = linregress(np.log(k), np.log(pk))
print slope, intercept

# Find u0 and kappa from slope and intercept
u0= np.exp(-intercept/slope)
kappa = -slope
print u0, kappa

In [ ]:
# Compare power-law fit to actual power-spectrum. We really only trust smaller scales.
plt.plot(k,pk)
plt.plot(k,(k/u0)**-kappa)
plt.xscale('log')
plt.yscale('log')